In [1]:
using PyPlot
using JuMP, Ipopt
using Combinatorics

In [2]:
include("../src/model.jl");
include("../src/diagnostics.jl");
include("../src/optimization.jl");
include("../src/plotting.jl");
include("../src/defaults.jl");

# Optimizating the Global Climate Action Policy Portfolio

In [3]:
model_og = ClimateModel();
max_deployment_og = Dict("mitigate"=>1., "remove"=>1., "geoeng"=>1., "adapt"=>1. /3.)

Dict{String,Float64} with 4 entries:
  "geoeng"   => 1.0
  "mitigate" => 1.0
  "remove"   => 1.0
  "adapt"    => 0.333333

In [4]:
controls = collect(keys(max_deployment_og));

In [5]:
control_permutations = combinations(controls) |> collect

15-element Array{Array{String,1},1}:
 ["geoeng"]                               
 ["mitigate"]                             
 ["remove"]                               
 ["adapt"]                                
 ["geoeng", "mitigate"]                   
 ["geoeng", "remove"]                     
 ["geoeng", "adapt"]                      
 ["mitigate", "remove"]                   
 ["mitigate", "adapt"]                    
 ["remove", "adapt"]                      
 ["geoeng", "mitigate", "remove"]         
 ["geoeng", "mitigate", "adapt"]          
 ["geoeng", "remove", "adapt"]            
 ["mitigate", "remove", "adapt"]          
 ["geoeng", "mitigate", "remove", "adapt"]

In [6]:
results = Dict()

for control_permutation in control_permutations    
    # Assert that some controls not be deployed
    max_deployment = copy(max_deployment_og)
    for control in controls
        if ~(control in control_permutation)
            max_deployment[control] = 0
        end
    end
    
    model = ClimateModel();
    @time m = optimize_controls!(model, max_deployment = max_deployment);
    
    results[control_permutation] = Dict("NPV" => discounted_total_control_cost(model), "status" => raw_status(m))
    
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded
 36.589800 seconds (111.33 M allocations: 6.323 GiB, 5.46% gc time)
Solve_Succeeded
  0.037723 seconds (94.28 k allocations: 4.051 MiB)
Solve_Succeeded
  0.066116 seconds (182.91 k allocations: 5.711 MiB)
Infeasible_Problem_Detected
  0.989484 seconds (1.55 M allocations: 28.428 MiB, 2.69% gc time)
Solve_Succeeded
  0.049137 seconds (89.75 k allocations: 4.016 MiB)
Solve_Succeeded
  0.028723 seconds (71.90 k allocations: 3.693 MiB)
Solve_Succeeded
  0.036886 seconds (107.20 k allocations: 4.293 MiB)
Solve_Succeeded
  0.055833 seconds (115.51 k allocations: 4.482 MiB)
Solve_Succeeded
  0.04

In [7]:
min_value = Inf
for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded") & (value["NPV"] < min_value)
        min_value = value["NPV"]
    end
end

for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded")
        abs_value = Int64(round((value["NPV"])))
        rel_value = Int64(round((value["NPV"]/min_value - 1.)*100))
        print("$index, $abs_value, $rel_value%\n")
    else
        print("$index has no solution.")
    end
end


["geoeng", "mitigate", "remove"], 56, 2%
["remove"], 262, 372%
["mitigate", "remove"], 62, 11%
["mitigate"], 90, 62%
["geoeng", "mitigate", "remove", "adapt"], 55, 0%
["geoeng", "adapt"], 346, 524%
["geoeng", "mitigate", "adapt"], 76, 38%
["mitigate", "remove", "adapt"], 60, 9%
["geoeng"], 459, 727%
["geoeng", "remove", "adapt"], 152, 175%
["mitigate", "adapt"], 87, 58%
["geoeng", "mitigate"], 79, 42%
["remove", "adapt"], 225, 305%
["geoeng", "remove"], 170, 207%
["adapt"] has no solution.